# Best practice: Set root directory to the root of the project


In [1]:
%load_ext autoreload
%autoreload 2
from constants import ROOT_DIR
import os
import sys
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

# Load Imports and Configs

In [ ]:
from configs.data_config import DataConfig
from configs.processing_config import DataProcessingConfig
import pandas as pd
from src.feature_processing.process_features import ProcessFeatures

data_config = DataConfig()
processing_config = DataProcessingConfig()

# Load Processed Data

In [37]:
users = pd.read_parquet(data_config.processed_users.file_path)
movies = pd.read_parquet(data_config.processed_movies.file_path)
ratings = pd.read_parquet(data_config.processed_ratings.file_path)

users.head()
movies.head()
ratings.head()

,user_id,movie_id,rating,timestamp,interaction_num,qid,label
0,1,1193,5,978300760,42.0,0,1
1,1,661,3,978302109,23.0,1,1
2,1,914,3,978301968,28.0,2,1
3,1,3408,4,978300275,47.0,3,1
4,1,2355,5,978824291,4.0,4,1


# Process Features for Training

In [35]:
pf_object = ProcessFeatures(users, movies, ratings, processing_config)
pf_object.process_for_training()

2025-05-04 16:04:35.718 | INFO     | src.feature_processing.process_features:combine_features:34 - Combining features to Ratings
2025-05-04 16:04:39.981 | INFO     | src.feature_processing.process_features:combine_features:58 - Completed combining features to Ratings
2025-05-04 16:04:39.985 | INFO     | src.feature_processing.process_features:train_test_split:62 - Creating train and val data for modelling
2025-05-04 16:04:40.702 | INFO     | src.feature_processing.process_features:train_test_split:82 - Completed creating train and val data for modelling
2025-05-04 16:04:40.703 | INFO     | src.feature_processing.process_features:numerical_feature_processing_for_training:86 - Processing numerical features
2025-05-04 16:04:42.449 | INFO     | src.feature_processing.process_features:numerical_feature_processing_for_training:95 - Completed processing numerical features
2025-05-04 16:04:42.451 | INFO     | src.feature_processing.process_features:categorical_feature_processing_for_training:9

# Save Processed Data for training

In [36]:
pf_object.train_data.to_parquet(data_config.train.file_path)
pf_object.val_data.to_parquet(data_config.val.file_path)
pf_object.test_data.to_parquet(data_config.test.file_path)

# Test process features for inference

In [38]:
pf_object = ProcessFeatures(users, movies, ratings,processing_config)
pf_object_loaded = pf_object.load()
pf_object_loaded.process_for_inference()
master_data = pf_object_loaded.master_data

2025-05-04 16:06:12.135 | INFO     | src.feature_processing.process_features:load:16 - Loaded
2025-05-04 16:06:12.138 | INFO     | src.feature_processing.process_features:combine_features:34 - Combining features to Ratings
2025-05-04 16:06:16.497 | INFO     | src.feature_processing.process_features:combine_features:58 - Completed combining features to Ratings
2025-05-04 16:06:16.499 | INFO     | src.feature_processing.process_features:numerical_feature_processing_for_inference:113 - Processing numerical features
2025-05-04 16:06:18.135 | INFO     | src.feature_processing.process_features:numerical_feature_processing_for_inference:115 - Completed processing numerical features
2025-05-04 16:06:18.141 | INFO     | src.feature_processing.process_features:categorical_feature_processing_for_inference:119 - Processing categorical features
2025-05-04 16:06:19.467 | INFO     | src.feature_processing.process_features:categorical_feature_processing_for_inference:121 - Completed processing categor

In [41]:
pf_object = ProcessFeatures(users, movies, ratings, processing_config)
pf_object_loaded = pf_object.load()
pf_object_loaded.process_for_inference()
pf_object_loaded.master_data.to_parquet(data_config.inference_model_input.file_path)